In [11]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#model
#loss and optimizer
#training loop

#preparing data - taking data from sklearn datasets
bc=datasets.load_breast_cancer()
X,y=bc.data,bc.target   # X contains features, y contains labels 0 or 1

n_samples, n_features=X.shape
print(n_samples, n_features)
#splitting data in sets
# 80% training and 20% testing
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1234)

#scaling features
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

#converting numpy arrays to PyTorch tensors of type float32
X_train=torch.from_numpy(X_train.astype(np.float32))
X_test=torch.from_numpy(X_test.astype(np.float32))
y_train=torch.from_numpy(y_train.astype(np.float32))
y_test=torch.from_numpy(y_test.astype(np.float32))

#reshape y tensor
#Reshape y_train and y_test to be column vectors
y_train=y_train.view(y_train.shape[0],1)
y_test=y_test.view(y_test.shape[0],1)

#setting up model

class LogisticRegression(nn.Module):
    def __init__(self,n_input_features):
        super(LogisticRegression,self).__init__()
        self.linear=nn.Linear(n_input_features,1)

    def forward(self,x):
        #sigmoid - logistic function
        y_pred=torch.sigmoid(self.linear(x))
        return y_pred   #returns 0 or 1

model=LogisticRegression(n_features)   #30 input features, 1 output feature

#loss and optimizer
criterion=nn.BCELoss()            #binary cross entropy loss
optimizer=torch.optim.SGD(model.parameters(),lr=0.01)  #learning rate is 0.01    #using stochastic gradient descent

#training loop
num_epochs=100
for epoch in range(num_epochs):
    #forward pass and loss
    y_pred=model(X_train)
    loss=criterion(y_pred,y_train)

    #backward pass and optimizer step
    loss.backward()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch + 1} - Before updating: Weights: {model.linear.weight.data.numpy()}')

    #update weight
    optimizer.step()

    # Print the values of theta (weights) after updating
    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch + 1} - After updating: Weights: {model.linear.weight.data.numpy()}')


    #empty gradient
    optimizer.zero_grad()

    if(epoch+1)%10==0:    #after every 10th step
        print(f'epoch:{epoch+1}, loss={loss.item():.4f}')

with torch.no_grad():
    y_predicted=model(X_test)
    #if y is greater than 0.5 than 1 otherwise 0
    y_predicted_cls=y_predicted.round()
    #calculating accuracy
    acc=y_predicted_cls.eq(y_test).sum()/float(y_test.shape[0])
    print()
    print(f'accuracy={acc.item():.4f}')




569 30
Epoch 10 - Before updating: Weights: [[-0.17073406 -0.1711105  -0.13568045 -0.02915274 -0.16110763 -0.09371217
   0.0763087  -0.02072659 -0.16975053 -0.0309427  -0.00556845 -0.03620943
   0.05817804  0.1473701   0.16461124  0.1604254   0.10457321 -0.06082732
  -0.11116552 -0.09127568 -0.16230337 -0.06632227 -0.20284383  0.07208515
  -0.12065051 -0.06398452 -0.04981894  0.11550106  0.16690008 -0.01826466]]
Epoch 10 - After updating: Weights: [[-0.17301682 -0.17222755 -0.13801351 -0.03139529 -0.16219965 -0.09567025
   0.07396628 -0.0232381  -0.17072085 -0.03096887 -0.00750027 -0.03602823
   0.05626854  0.14543271  0.16457392  0.15931483  0.10358009 -0.06227916
  -0.11111563 -0.09168877 -0.16476777 -0.06757452 -0.20533884  0.06973836
  -0.12209271 -0.06593622 -0.05205654  0.11287782  0.16553918 -0.01941744]]
epoch:10, loss=0.4870
Epoch 20 - Before updating: Weights: [[-0.19135179 -0.18135822 -0.15672535 -0.04937538 -0.1706881  -0.11100165
   0.0553793  -0.04327925 -0.17825574 -0.03